<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_fhir_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 09/2022**

---

**Funktion**

FHIR - Abfragen und Suchen (nativ und mittels fhirclient)

---



**Referenzen**

https://github.com/Alvearie/FHIR-from-Jupyter/blob/main/Bonus%20Notebook%20-%20Location%20Search.ipynb 


---



In [1]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [2]:
# Define Base URL
url = "http://hapi.fhir.org/baseR4/" # Open HAPI FHIR Server
#url = "https://server.fire.ly/R4/" # Open Firely Server
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

In [ ]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
IFrame('https://www.hl7.org/fhir/patient.html#search', width=1200, height=500)

In [ ]:
# Abfrage nach Name UND Geburtsdatum

# SEARCH PARAM and FORMAT (as documented in FHIR R4)
# birthdate = date
# name = string

req = url + "Patient?" + "birthdate" + "=" + "1950-01-01" + "&" + "name" + "=" + "MrXXX"
print(req)
response = requests.request("GET", req, headers=headers)

print(response.text)

In [ ]:
# diverse Abfrage
# MrXXX
patient_id = str(7168516)
# Get Condition of this patient
req = url + "Condition/" + "?patient=" + patient_id
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Alle Patienten mit einer gewissen Observation (Heart Rate)
req = url + "/Patient?_has:Observation:patient:code=8867-4"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# alle Patienten mit Namen
req = url + "Patient?name=Alex"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Anzahl der Patienten
req = url + "Patient?_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

In [ ]:
# Anzahl der Patienten
req = url + "Patient?_total=accurate"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [23]:
# Verteilung des Geschlechts
req = url + "Patient" + '?' + 'gender=male' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("male: " + str(response.json().get('total')))
req = url + "Patient" + '?' + 'gender=female' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("female: " + str(response.json().get('total')))
req = url + "Patient" + '?' + 'gender=missing=true' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("missing: " + str(response.json().get('total')))

male: 142792
female: 64011
missing: 0


In [ ]:
# String Search
#req = url + 'Patient' + '?' + 'family=MrXXX' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:exact=MrXXX' + '&_elements=name'
req = url + 'Patient' + '?' + 'family:contains=Mr' + '&_elements=name'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Date Search
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"
# some servers support ap(approximately equal) as well, although the spec lets the server decide exactly what that means...
req = url + 'Patient' + '?' + 'birthdate=1984' + '&_elements=birthDate'
#req = url + 'Patient' + '?' + 'birthdate=eb1984' + '&_elements=birthDate'
#req = url + 'Patient' + '?' + 'birthdate=sa1984' + '&_elements=birthDate'
#req = url + 'Patient' + '?' + 'birthdate=ap1984' + '&_elements=birthDate'
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Reference search
#response = get(base + '/Patient?general-practitioner:missing=false&_elements=generalPractitioner,link,managingOrganization&_count=1')

In [ ]:
#req = url + "Patient$export"

In [ ]:
# Search for all Patients with Diabetes Mellitus

# Use the _has parameter
# Patient is selected based on Condition
# Condition references the patient (thus links the inforamtion)
# Condition has the search parameter patient
# The patient is the subject of the condition, which itself has a code element
# We look for Patients with Diabetes Mellitus

req = url + "Patient?_has:Condition:patient:code=E10.65"
response = requests.request("GET", req, headers=headers)
print(response.text)

In [ ]:
# Location Search

#  